In [18]:
import ee 
import geemap 

ee.Initialize()

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [19]:
#load the asset and dataset 
assetId = 'users/bornToBeAlive/aoi_PU' 
datasetId = 'UMD/hansen/global_forest_change_2019_v1_7' 

dataset = ee.Image(datasetId)
aoi = ee.FeatureCollection(assetId)

#clip the dataset on the aoi 
clip_dataset = dataset.clip(aoi)

Map.centerObject(aoi, 8)

In [20]:
#create a composite band bassed on the user threshold 
threshold = 30

calc = "gfc = (A<={0})*((C==1)*50 + (C==0)*30) + " #Non forest 
calc += "(A>{0})*(C==1)*(B>0)*51 + "         #gain + loss 
calc += "(A>{0})*(C==1)*(B==0)*50 + "        #gain                                             
calc += "(A>{0})*(C==0)*(B>0)*B + "          #loss
calc += "(A>{0})*(C==0)*(B==0)*40"           #stable forest

calc = calc.format(threshold)

print(calc)

gfc = (A<=30)*((C==1)*50 + (C==0)*30) + (A>30)*(C==1)*(B>0)*51 + (A>30)*(C==1)*(B==0)*50 + (A>30)*(C==0)*(B>0)*B + (A>30)*(C==0)*(B==0)*40


In [21]:
bands = {
    'A': clip_dataset.select('treecover2000'),
    'B': clip_dataset.select('lossyear').unmask(0), #be carefull the 0 values are now masked
    'C': clip_dataset.select('gain'),
}

gfc = clip_dataset.expression(calc,bands)

In [22]:
#Define an SLD style of discrete intervals to apply to the image.
sld_intervals = '<RasterSymbolizer>' 
sld_intervals += '<ColorMap type="intervals" extended="false" >' 
sld_intervals += '<ColorMapEntry color="#000000" quantity="0" label="no data"/>' 
sld_intervals += '<ColorMapEntry color="#F9F200" quantity="1" label="loss-2001"/>' 
sld_intervals += '<ColorMapEntry color="#DFF800" quantity="2" label="loss-2002"/>' 
sld_intervals += '<ColorMapEntry color="#EDD700" quantity="3" label="loss-2003"/>' 
sld_intervals += '<ColorMapEntry color="#E7C900" quantity="4" label="loss-2004"/>' 
sld_intervals += '<ColorMapEntry color="#E0BC00" quantity="5" label="loss-2005"/>' 
sld_intervals += '<ColorMapEntry color="#DAAE00" quantity="6" label="loss-2006"/>' 
sld_intervals += '<ColorMapEntry color="#D4A100" quantity="7" label="loss-2007"/>' 
sld_intervals += '<ColorMapEntry color="#CE9400" quantity="8" label="loss-2008"/>' 
sld_intervals += '<ColorMapEntry color="#C88600" quantity="9" label="loss-2009"/>' 
sld_intervals += '<ColorMapEntry color="#C27900" quantity="10" label="loss-2010"/>' 
sld_intervals += '<ColorMapEntry color="#BC6B00" quantity="11" label="loss-2011"/>' 
sld_intervals += '<ColorMapEntry color="#B65E00" quantity="12" label="loss-2012"/>' 
sld_intervals += '<ColorMapEntry color="#B05100" quantity="13" label="loss-2013"/>' 
sld_intervals += '<ColorMapEntry color="#AA4300" quantity="14" label="loss-2014"/>' 
sld_intervals += '<ColorMapEntry color="#A33600" quantity="15" label="loss-2015"/>' 
sld_intervals += '<ColorMapEntry color="#9D2800" quantity="16" label="loss-2016"/>' 
sld_intervals += '<ColorMapEntry color="#971B00" quantity="17" label="loss-2017"/>' 
sld_intervals += '<ColorMapEntry color="#910D00" quantity="18" label="loss-2018"/>' 
sld_intervals += '<ColorMapEntry color="#8B0000" quantity="19" label="loss-2019"/>' 
sld_intervals += '<ColorMapEntry color="#D3D3D3" quantity="30" label="non forest"/>' 
sld_intervals += '<ColorMapEntry color="#006400" quantity="40" label="stable forest"/>' 
sld_intervals += '<ColorMapEntry color="#800080" quantity="51" label="gain + loss"/>' 
sld_intervals += '</ColorMap>'
sld_intervals += '</RasterSymbolizer>'

Map.addLayer(gfc.select('gfc').sldStyle(sld_intervals), {}, 'gfc');

clip_map = gfc.select('gfc')
clip_map.getInfo()

{'type': 'Image',
 'bands': [{'id': 'gfc',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 476},
   'crs': 'EPSG:4326',
   'crs_transform': [0.00025, 0, -180, 0, -0.00025, 80]}]}

In [23]:
#compute area
import pandas as pd
from utils import parameters as pm

res = 30

hist = clip_map.reduceRegion(**{
  'reducer': ee.Reducer.autoHistogram(),
  'geometry': aoi.geometry(),
  'scale': res,
  'maxPixels': 1e12
})

hist = pd.DataFrame(hist.getInfo()['gfc'])

#add column name
hist.columns= ['code', 'pixels']

codes = [30, 40, 50, 51] + [i for i in range(1, pm.getMaxYear() + 1)]
codes = [str(i) for i in codes]

#dropping the 0 lines (a priori tout sauf les trucs pertinents)
hist = hist[hist['code'].isin(codes)]


#construct the surface values
hist['area'] = hist['pixels']*res*res/10000

#construct the labels
label = pm.getMyLabel()
label.pop(0) #remove the no-data (it will be removed when it'll work)
hist['class'] = label

hist

,code,pixels,area,class
0,1,2.506836e+04,2.256152e+03,loss_2001
1,2,6.415645e+04,5.774080e+03,loss_2002
2,3,1.016323e+04,9.146905e+02,loss_2003
3,4,4.135299e+04,3.721769e+03,loss_2004
4,5,3.870298e+04,3.483269e+03,loss_2005
5,6,2.437811e+04,2.194030e+03,loss_2006
6,7,6.607062e+04,5.946356e+03,loss_2007
7,8,4.070174e+04,3.663157e+03,loss_2008
8,9,8.434706e+04,7.591235e+03,loss_2009
9,10,5.750842e+04,5.175757e+03,loss_2010


In [ ]:
task_config = {
    'image':clip_map,
    'description':'test',
    'scale': 30,
    'region':aoi.geometry(),
    'maxPixels': 1e12
}

task = ee.batch.Export.image.toDrive(**task_config)
task.start()

In [26]:
import glob
task_name = 'PU' + '_{}_gfc_map'.format(threshold)
path = pm.getGfcDir() + task_name + '*.tif'
print(path)
glob.glob(path)

/home/prambaud/gfc_wrapper_results/gfc/PU_30_gfc_map*.tif


['/home/prambaud/gfc_wrapper_results/gfc/PU_30_gfc_map.tif']